In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
import SimpleITK as sitk
import pydicom
from pydicom.tag import Tag
import sys
import csv
import pydicom as pdcm
import pandas as pd

In [3]:
pathtoPET = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0846557\0/Vision PET Earl/"
files = os.listdir(pathToPET)
ds = pydicom.dcmread(pathtoPET)

ValueError: embedded null character

In [3]:
class DicomHeader:
    def __init__(self, ds):
        self.seriesDesc = ds.SeriesDescription
        try:
            self.corrImg = ds.CorrectedImage
        except: self.corrImg = None
        try: 
            self.petUnits = ds.Units
        except: self.petUnits = None
        try:
            self.seriesType = ds.SeriesType
        except: self.seriesType = None
        try:
            self.countsSource = ds.CountsSource
        except: self.countsSource = None
        try:
            self.randCorrMethod = ds.RandomsCorrectionMethod
        except: self.randCorrMethod = None
        try:
            self.attCorrMeth = ds.AttenuationCorrectionMethod
        except: self.attCorrMeth = None
        try:
            self.decayCorr = ds.DecayCorrection
        except: self.decayCorr = None
        try: 
            self.reccMethod = ds.ReconstructionMethod
        except: self.reccMethod = None
        try:
            self.scaterCorrMeth = ds.ScatterCorrectionMethod
        except: self.scaterCorrMeth =None
        self.imgType = ds.ImageType
        self.institution = ds.InstitutionName
        try:
            self.codeMeaning1 = ds.RelatedSeriesSequence[0][0x0040,0xa170][0].CodeMeaning
        except: self.codeMeaning1 = None    
        try:
            self.codeMeaning2 = ds.RelatedSeriesSequence[0][0x0040,0xa170][1].CodeMeaning
        except: self.codeMeaning2 = None
        

In [4]:
new_columns = ["seriesDesc","corrImg","petUnits","seriesType","countsSource","randCorrMethod","attCorrMeth","decayCorr","reccMethod","scaterCorrMeth","imgType","institution","codeMeaning1","codeMeaning2"]
for col_name in new_columns:
    df_new = df_new.assign(**{col_name: 0})
df_new

,ID,CT,PET,Date,Name,seriesDesc,corrImg,petUnits,seriesType,countsSource,randCorrMethod,attCorrMeth,decayCorr,reccMethod,scaterCorrMeth,imgType,institution,codeMeaning1,codeMeaning2
0,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,17420,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,9/4/2017,PET WB Earl,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,9898346,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,6/28/2017,PET WB Uncorrected,0,0,0,0,0,0,0,0,0,0,0,0,0,0
760,9915436,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,2/6/2019,PET WB_EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
761,9920297,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/4/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
762,9935146,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,11/19/2019,Vision PET Earl,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
#read PET Corrected 
for count in range(len(df_new)):  
    pet_path = df_new['PET'][count]
    listofDCMs = os.listdir(pet_path)
    for singledcm in listofDCMs:
        ds = pydicom.dcmread(os.path.join(pet_path,singledcm),stop_before_pixels=True)
        if ds.Modality =='PT':
            header_class = DicomHeader(ds)
            break

    for attr in vars(header_class):
        df_new.loc[count,attr] = str(getattr(header_class, attr))

In [7]:
saveCSV = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/PETasMarch27_FullW_withDCMHEader.csv"
df_new.to_csv(saveCSV, index=False)

In [8]:
df_new

,ID,CT,PET,Date,Name,seriesDesc,corrImg,petUnits,seriesType,countsSource,randCorrMethod,attCorrMeth,decayCorr,reccMethod,scaterCorrMeth,imgType,institution,codeMeaning1,codeMeaning2
0,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,PET WB EARL,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC_CT_Body",START,PSF+TOF 3i21s,Model-based,"['ORIGINAL', 'PRIMARY']",UMCG,Same Anatomy,For Attenuation Correction
1,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,PET_E,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC_CT_Body",START,PSF+TOF 3i21s,Model-based,"['DERIVED', 'PRIMARY']",University Medical Center Groningen,None,None
2,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,PET_E,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC_CT_Body",START,PSF+TOF 3i21s,Model-based,"['DERIVED', 'PRIMARY']",University Medical Center Groningen,None,None
3,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,PET WB EARL,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC_CT_Body",START,PSF+TOF 3i21s,Model-based,"['ORIGINAL', 'PRIMARY']",UMCG,Same Anatomy,For Attenuation Correction
4,17420,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,9/4/2017,PET WB Earl,PET WB Earl,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC CT Body 2.0 HD_FoV",START,PSF+TOF 3i21s,"Model-based, relative scatter scaling","['ORIGINAL', 'PRIMARY']",UMCG PC1,Same Anatomy,For Attenuation Correction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,9898346,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,6/28/2017,PET WB Uncorrected,PET WB Uncorrected,"['NORM', 'DTIM', 'DECY']",PROPCPS,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,None,START,OSEM3D+TOF 3i21s,None,"['ORIGINAL', 'PRIMARY']","Treant Zorggroep, Nucleaire Geneeskunde",Same Anatomy,None
760,9915436,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,2/6/2019,PET WB_EARL,PET WB_EARL,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC CT WB LD 5.0 HD_FoV",START,OSEM3D+TOF 2i21s,"Model-based, relative scatter scaling","['ORIGINAL', 'PRIMARY']",,Same Anatomy,For Attenuation Correction
761,9920297,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/4/2015,PET WB EARL,PET WB EARL,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,AC_CT_Body",START,PSF+TOF 3i21s,Model-based,"['ORIGINAL', 'PRIMARY']",UMCG,Same Anatomy,For Attenuation Correction
762,9935146,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,11/19/2019,Vision PET Earl,Vision PET Earl,"['NORM', 'DTIM', 'ATTN', 'SCAT', 'DECY', 'RAN']",BQML,"['WHOLE BODY', 'IMAGE']",EMISSION,DLYD,"measured,Vision AC CT",START,PSF+TOF 4i5s,"Model-based, relative scatter scaling","['ORIGINAL', 'PRIMARY']",UMCG,Same Anatomy,For Attenuation Correction
